# Homework 4

Due date: May 26, 2024

### Submission instructions: 
- __Autograder will not be used for scoring, but you still need to submit the python file converted from this notebook (.py) and the notebook file (.ipynb) to the code submission window.__ 
To convert a Jupyter Notebook (`.ipynb`) to a regular Python script (`.py`):
  - In Jupyter Notebook: File > Download as > Python (.py)
  - In JupyterLab: File > Save and Export Notebook As... > Executable Script
  - In VS Code Jupyter Notebook App: In the toolbar, there is an Export menu. Click on it, and select Python script.
- Submit `hw4.ipynb` and `hw4.py` on Gradescope under the window "Homework 4 - code". Do **NOT** change the file name.
- Convert this notebook into a pdf file and submit it on Gradescope under the window "Homework 4 - PDF". Make sure all your code and text outputs in the problems are visible. 


### General instructions: 

In this homework, we will use pandas to build a cohort of ICU stays and visualize the results from the MIMIC-IV dataset, which you did for Homework 3 in BIOSTAT 203B. 

For processing the Parquet files, one other option is [`polars`](https://pola.rs/). The package is designed for rapid analysis of data frames, possibly larger than memory, with pandas-like syntax, Apache Arrow-based data representation and the Rust language as its backend. Syntax is similar to what you have used for `pyarrow`. You are allowed to use any method you like for analyzing data, but use of `pyarrow`, `duckdb`, or `polars` is certainly recommended for larger files to save your memory and time. (_Hint_: If you want to try `polars`, look into `scan_parquet()` and `LazyFrame`. The package `polars` supports lazy evaluation similar to what you have seen in the R `arrow` package.)

For visualization, you may use packages `matplotlib`, `seaborn`, and/or `plotly`. The use of `plotnine` is not allowed.

Please run the code below to show your system information:

In [1]:
import json
import platform

import psutil


def get_system_info():
    try:
        info = {}
        info["platform"] = platform.system()
        info["platform-release"] = platform.release()
        info["platform-version"] = platform.version()
        info["architecture"] = platform.machine()
        info["processor"] = platform.processor()
        info["ram"] = str(round(psutil.virtual_memory().total / (1024.0**3))) + " GB"
        for k, v in info.items():
            print(f"{k}:\t{v}")
    except Exception as e:
        logging.exception(e)

In [2]:
get_system_info()

platform:	Linux
platform-release:	6.5.0-28-generic
platform-version:	#29~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Apr  4 14:39:20 UTC 2
architecture:	x86_64
processor:	x86_64
ram:	63 GB


In [3]:
import datetime as dt
import os
import re

import duckdb
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import polars as pl
import seaborn as sns
from matplotlib.collections import PolyCollection

## Problem 1. Visualizing patient trajectory

Visualizing a patient’s encounters in a health care system is a common task in clinical data analysis. In this question, we will visualize a patient’s ADT (admission-discharge-transfer) history and ICU vitals in the MIMIC-IV data.

### (A). ADT history
A patient’s ADT history records the time of admission, discharge, and transfer in the hospital. This figure shows the ADT history of the patient with subject_id `10001217` in the MIMIC-IV data. The x-axis is the calendar time, and the y-axis is the type of event (ADT, lab, procedure). The color of the line segment represents the care unit. The size of the line segment represents whether the care unit is an ICU/CCU. The crosses represent lab events, and the shape of the dots represents the type of procedure. The title of the figure shows the patient’s demographic information and the subtitle shows top 3 diagnoses. Try to create a figure similar to the below:

<figure>
  <img src="https://raw.githubusercontent.com/ucla-biostat-203b/2024winter/main/hw/hw3/10001217_adt.png" style="width:600px">
</figure>


Your figure does not need to be the same, but all the information in this figure should be reasonably arranged in your figure. Hint: consider using `dodge` keyword arguments of seaborn to do something similar to `jitter` of `ggplot2`. 



Hint: We need to pull information from data files `patients.csv.gz`, `admissions.csv.gz`, `transfers.csv.gz`, `labevents.csv.gz`, `procedures_icd.csv.gz`, `diagnoses_icd.csv.gz`, `d_icd_procedures.csv.gz`, and `d_icd_diagnoses.csv.gz`. For the big file `labevents.csv.gz`, use the Parquet file you generated in Homework 3. More information is available in later problems.

For reproducibility, make the Parquet file available at the current working directory, for example, by a symbolic link. Make your code reproducible using relative path.

Do a similar visualization for the patient `10013310`.

### ANSWER 1A

#### Documentation

`Parquet` is a more preferred storage format than `csv.gz`; therefore the function below converts all of the files within a directory that are `csv.gz` to parquet using duckdb and os library. The OS library is used to list the filenames within a directory and DuckDB is used to execute a SQL script that reads in the csv.gz to a table and copies that table to parquet.

In [22]:
def csvgz_to_parquet(dir_list: list[str]) -> None:
    """
    Convert all csv.gz files in a directory list to parquet files.
    All columns are dtype string.
    ---
    Args:
        dir_list: A list of the realtive or absolute directory paths.
    Return:
        None
    """
    # loop through user provided list
    for path in dir_list:
        # Loop through all files in directory
        for filename in os.listdir(path):
            # Ignore dot-files
            if filename.startswith("."):
                continue
            # if filename ends with ".csv.gz"
            if filename.endswith(".csv.gz"):
                # path to .csv.gz
                file_path = os.path.join(path, filename)
                # parquet name
                pq_name = filename.split(".")[0] + ".parquet"
                # parquet save path
                pq_path = os.path.join(path, pq_name)
                print(file_path, " to ", pq_path)  # Verbose step
                # Read in the compressed file
                csvgz = pl.read_csv(
                    file_path,  # .csv.gz file path
                    infer_schema_length=0,  # Set All columns to type string
                    null_values="___",  # Set ___ to null
                )
                # Write the file as a parquet
                csvgz.write_parquet(pq_path)


def parquet_to_lazydict(dir_path: str) -> dict[str, pl.LazyFrame]:
    """
    Read in all parquet files in a directory with Polars as LazyFrames.
    ---
    Args:
        dir_path: A string of the realtive or absolute directory path.
    Return:
        lazy_dict: A dictionary of LazyFrames.
    """
    # Initialize lazyframe dictionary
    lazy_dict = dict()
    # Loop through every file in directory
    for file in os.listdir(dir_path):
        # Select files that end with parquet
        if file.endswith(".parquet"):
            # Path to each parquet
            name = file.split(".")[0]
            file_path = os.path.join(dir_path, file)
            # append each entry to lazy_dict
            # key is file name and value is lazyframe
            lazy_dict[name] = pl.scan_parquet(file_path)
    return lazy_dict


def lazy_to_df(lazy_dict: dict[str, pl.LazyFrame], prefix: str) -> pl.DataFrame:
    """
    Convert pl.LazyFrames to pl.DataFrames in a dictionary where the key starts
    with the prefix.
    ---
    Args:
        lazy_dict: Dictionary of Polars LazyFrames
        prefix: String and prefix of key in dictionary
    Return:
        df_dict: A dictionary of Polars DataFrames
    """
    df_dict = dict()  # init dict
    # Loop through all values in lazyframe dictionary
    for key, value in lazy_dict.items():
        # Filter LazyFrames by keys that start with the prefix
        if key.startswith(prefix):
            # Convert pl.LazyFrame to pl.DataFrame and Save to dictionary
            df_dict[key] = value.collect()
    return df_dict


def sid_admissions(lazy_dict: dict[str, pl.LazyFrame], sid: str) -> dict:
    """
    Filter rows, select columns, and assign dtypes to the Admissions dataset.
    ---
    Args: 
        lazy_dict: A dictionary of Polars LazyFrames
        sid: A string of digits corresponding to the Subject ID.
    Return:
        lazy_dict: A dictionary of Polars LazyFrames and a processed plot table.
    """
    lazy_dict["plot_admissions"] = (
        # Admissions Dataset
        lazy_dict["admissions"]
        # filter by sid
        .filter(pl.col("subject_id") == sid)
        # select columns to keep
        .select(
            [
                "subject_id",
                "hadm_id",
                "admittime",
                "dischtime",
                "deathtime",
                "admission_type",
                "admit_provider_id",
                "admission_location",
                "discharge_location",
                "insurance",
                "language",
                "marital_status",
                "race",
                "edregtime",
                "edouttime",
            ]
        )
        # Change column dtype
        .cast(
            {
                "subject_id": pl.Int64,
                "hadm_id": pl.Int64,
                "admission_type": pl.String,
                "admit_provider_id": pl.String,
                "admission_location": pl.String,
                "discharge_location": pl.String,
                "insurance": pl.String,
                "language": pl.String,
                "marital_status": pl.String,
                "race": pl.String,
            }
        )
        # Change column dtype to Date
        .with_columns(
            pl.col("admittime").str.to_datetime("%Y-%m-%d %H:%M:%S"),
            pl.col("dischtime").str.to_datetime("%Y-%m-%d %H:%M:%S"),
            pl.col("deathtime").str.to_datetime("%Y-%m-%d %H:%M:%S"),
            pl.col("edregtime").str.to_datetime("%Y-%m-%d %H:%M:%S"),
            pl.col("edouttime").str.to_datetime("%Y-%m-%d %H:%M:%S"),
        )
    )
    return lazy_dict


def sid_patient(lazy_dict: dict[str, pl.LazyFrame], sid: str) -> dict:
    """
    Filter rows, select columns, and assign dtypes to the Patient dataset.
    ---
    Args: 
        lazy_dict: A dictionary of Polars LazyFrames
        sid: A string of digits corresponding to the Subject ID.
    Return:
        lazy_dict: A dictionary of Polars LazyFrames and a processed plot table.
    """
    lazy_dict["plot_patient"] = (
        # Patient Info Dataset
        lazy_dict["patients"]
        # Filter by sid
        .filter(pl.col("subject_id") == sid)
        # Change column dtype
        .cast(
            {
                "subject_id": pl.Int64,
                "anchor_age": pl.Int64,
                "anchor_year": pl.Int64,
                "dod": pl.Date,
            }
        )
    )
    return lazy_dict


def sid_diagnosis(lazy_dict: dict[str, pl.LazyFrame], sid: str) -> dict:
    """
    Filter rows, select columns, join item key, and assign dtypes 
    to the Diagnosis dataset.
    ---
    Args: 
        lazy_dict: A dictionary of Polars LazyFrames
        sid: A string of digits corresponding to the Subject ID.
    Return:
        lazy_dict: A dictionary of Polars LazyFrames and a processed plot table.
    """
    lazy_dict["plot_diagnosis"] = (
        # diagnosis dataset
        lazy_dict["diagnoses_icd"]
        # Filter by sid
        .filter(pl.col("subject_id") == sid)
        # Change column dtype
        .cast({"subject_id": pl.Int64, "hadm_id": pl.Int64, "seq_num": pl.Int64})
        # Join diagnosis key
        .join(lazy_dict["d_icd_diagnoses"], on="icd_code", how="left")
        # Convert short title from long title
        .with_columns(
            pl.col("long_title").str.extract(r"^[^,]*", 0).alias("short_title")
        )
    )
    return lazy_dict


def sid_labevents(lazy_dict: dict[str, pl.LazyFrame], sid: str) -> dict:
    """
    Filter rows, select columns, and assign dtypes to the Lab Events dataset.
    ---
    Args: 
        lazy_dict: A dictionary of Polars LazyFrames
        sid: A string of digits corresponding to the Subject ID.
    Return:
        lazy_dict: A dictionary of Polars LazyFrames and a processed plot table.
    """
    # list of itemids to filter by
    lab_itemid = [
        "50912",
        "50971",
        "50983",
        "50902",
        "50882",
        "51221",
        "51301",
        "50931",
    ]
    lazy_dict["plot_lab"] = (
        # Labevents dataset
        lazy_dict["labevents"]
        # select columns to keep
        .select(
            [
                "labevent_id",
                "subject_id",
                "hadm_id",
                "specimen_id",
                "itemid",
                "order_provider_id",
                "charttime",
                "storetime",
                "valuenum",
                "valueuom",
            ]
        )
        # Filter by sid and itemid
        .filter(pl.col("subject_id") == sid, pl.col("itemid").is_in(lab_itemid))
        # Change column dtype
        .cast(
            {
                "labevent_id": pl.Int64,
                "subject_id": pl.Int64,
                "hadm_id": pl.Int64,
                "specimen_id": pl.Int64,
                "itemid": pl.Int64,
                "valuenum": pl.Float64,
            }
        )
        # Convert dtype to datetime
        .with_columns(
            pl.col("charttime").str.to_datetime("%Y-%m-%d %H:%M:%S"),
            pl.col("storetime").str.to_datetime("%Y-%m-%d %H:%M:%S"),
        )
    )
    return lazy_dict


def sid_procedures(lazy_dict: dict[str, pl.LazyFrame], sid: str) -> dict:
    """
    Filter rows, select columns, join item key, and assign dtypes 
    to the Procedures dataset.
    ---
    Args: 
        lazy_dict: A dictionary of Polars LazyFrames
        sid: A string of digits corresponding to the Subject ID.
    Return:
        lazy_dict: A dictionary of Polars LazyFrames and a processed plot table.
    """
    # preparing procedure key
    lazy_dict["d_icd_procedures"] = lazy_dict["d_icd_procedures"].select(
        ["icd_code", "long_title"]
    )

    lazy_dict["plot_procedures"] = (
        # procedure dataset
        lazy_dict["procedures_icd"]
        # select columns to keep
        .select(["subject_id", "hadm_id", "seq_num", "chartdate", "icd_code"])
        # Filter by sid
        .filter(pl.col("subject_id") == sid)
        # Change column dtype
        .cast(
            {
                "subject_id": pl.Int64,
                "hadm_id": pl.Int64,
                "seq_num": pl.Int64,
                "icd_code": pl.String,
                "chartdate": pl.Date,
            }
        )
        # Join procedures key
        .join(lazy_dict["d_icd_procedures"], on="icd_code", how="left")
        # convert long title to short title 
        .with_columns(
            pl.col("long_title").str.extract(r"^[^,]*", 0).alias("short_title")
        )
    )
    return lazy_dict


def sid_transfer(lazy_dict: dict[str, pl.LazyFrame], sid: str) -> dict:
    """
    Filter rows, select columns, and assign dtypes to the Transfers dataset.
    ---
    Args: 
        lazy_dict: A dictionary of Polars LazyFrames
        sid: A string of digits corresponding to the Subject ID.
    Return:
        lazy_dict: A dictionary of Polars LazyFrames and a processed plot table.
    """
    lazy_dict["plot_transfer"] = (
        # transfers dataset
        lazy_dict["transfers"]
        # filter by sid
        .filter(pl.col("subject_id") == sid, pl.col("eventtype") != "discharge")
        # Change column dtype
        .cast(
            {
                "subject_id": pl.Int64,
                "hadm_id": pl.Int64,
                "transfer_id": pl.Int64,
                "eventtype": pl.String,
                "careunit": pl.String,
            }
        )
        .with_columns(
            # Convert string columns to date
            pl.col("intime").str.slice(0, 10).str.to_date(),
            pl.col("outtime").str.slice(0, 10).str.to_date(),
            # Add a bivariate critical care column if in ICU or CCU
            (
                pl.when(pl.col("careunit").str.contains("ICU|CCU"))
                .then(pl.lit("Critical_Care"))
                .otherwise(pl.lit(""))
                .alias("Critical_Care")
            ),
            # Add a column ADT with values of ADT
            (
                pl.when(pl.col("careunit").is_not_null())
                .then(pl.lit("ADT"))
                .otherwise(pl.lit(""))
                .alias("ADT")
            ),
        )
    )
    return lazy_dict


def sid_chartevents(lazy_dict: dict[str, pl.LazyFrame], sid: str) -> dict:
    """
    Filter rows, select columns, join item key, and assign dtypes 
    to the Chartevents dataset.
    ---
    Args: 
        lazy_dict: A dictionary of Polars LazyFrames
        sid: A string of digits corresponding to the Subject ID.
    Return:
        lazy_dict: A dictionary of Polars LazyFrames and a processed plot table.
    """
    # Chartevents item key
    lazy_dict["d_items"] = (
        lazy_dict["d_items"]
        # select columns to keep
        .select(["itemid", "label", "abbreviation", "unitname"])
        # convert dtype of columns
        .cast(
            {
                "itemid": pl.Int64,
                "label": pl.String,
                "abbreviation": pl.String,
                "unitname": pl.String,
            }
        )
    )

    # Patient chartevents
    lazy_dict["plot_chartevents"] = (
        # Chartevents dataset
        lazy_dict["chartevents"]
        # Filter by sid and itemid
        .filter(
            pl.col("subject_id") == sid,
            pl.col("itemid").is_in(['220045', '220181', '220179', '223761', '220210']),
        )
        # Select columns to keep
        .select(
            [
                "subject_id",
                "hadm_id",
                "stay_id",
                "charttime",
                "storetime",
                "itemid",
                "valuenum",
                "valueuom",
            ]
        )
        # Change column dtype
        .cast(
            {
                "subject_id": pl.Int64,
                "hadm_id": pl.Int64,
                "stay_id": pl.Int64,
                "itemid": pl.Int64,
                "valuenum": pl.Float64,
                "valueuom": pl.String,
            }
        )
        # Convert columns to datetime
        .with_columns(
            pl.col("charttime").str.to_datetime("%Y-%m-%d %H:%M:%S"),
            pl.col("storetime").str.to_datetime("%Y-%m-%d %H:%M:%S"),
        )
        # join item key
        .join(lazy_dict["d_items"], on="itemid", how="left")
    )
    return lazy_dict


def create_dicts(sid: int):
    """
    Combine functions to create 2 dictionaries, a Polars LazyFrame and DataFrame.
    ---
    Args:
        sid: An integer representing a subject ID.
    Returns:
        lazy_dict: A dictionary of LazyFrames
        df_dict: A dictionary of DataFrames
    """
    lazy_dict = parquet_to_lazydict("./")
    sid = str(sid)
    lazy_names = [key for key in lazy_dict]
    names = {
        'patients': lambda: sid_patient(lazy_dict, sid),
        'admissions': lambda: sid_admissions(lazy_dict, sid),
        'transfers': lambda: sid_transfer(lazy_dict, sid),
        'procedures_icd': lambda: sid_procedures(lazy_dict, sid),
        'diagnoses_icd': lambda: sid_diagnosis(lazy_dict, sid),
        'labevents': lambda: sid_labevents(lazy_dict, sid),
        'chartevents': lambda: sid_chartevents(lazy_dict, sid),
    }
    for key, fxn in names.items():
        if key in lazy_names:
            fxn()
        else:
            print(f"{key} not available in lazy_dict")
    df_dict = lazy_to_df(lazy_dict, "plot")
    return lazy_dict, df_dict

In [23]:
create_dicts(10013310)

({'transfers': <LazyFrame [7 cols, {"subject_id": String … "outtime": String}] at 0x760D32694250>,
  'diagnoses_icd': <LazyFrame [5 cols, {"subject_id": String … "icd_version": String}] at 0x760D326971D0>,
  'd_icd_diagnoses': <LazyFrame [3 cols, {"icd_code": String … "long_title": String}] at 0x760D32696E10>,
  'admissions': <LazyFrame [16 cols, {"subject_id": String … "hospital_expire_flag": String}] at 0x760D32696C10>,
  'chartevents': <LazyFrame [11 cols, {"subject_id": String … "warning": String}] at 0x760D32697E10>,
  'd_items': <LazyFrame [4 cols, {"itemid": Int64 … "unitname": String}] at 0x760D32695A50>,
  'procedures_icd': <LazyFrame [6 cols, {"subject_id": String … "icd_version": String}] at 0x760D32694C10>,
  'patients': <LazyFrame [6 cols, {"subject_id": String … "dod": String}] at 0x760D32697650>,
  'labevents': <LazyFrame [16 cols, {"labevent_id": String … "comments": String}] at 0x760D32696750>,
  'd_icd_procedures': <LazyFrame [2 cols, {"icd_code": String, "long_title"

#### Documentation

Bash commands below were used to create symbolic links to relavent datasets within working directory.

```bash
ln -sf /media/usb/mimic /home/wtmartinez/github/203c-hw/hw4/
ln -sf ./mimic/hosp/patients.parquet ./patients.parquet
ln -sf ./mimic/hosp/admissions.parquet ./admissions.parquet
ln -sf ./mimic/hosp/transfers.parquet ./transfers.parquet
ln -sf ./mimic/hosp/labevents.parquet ./labevents.parquet
ln -sf ./mimic/hosp/procedures_icd.parquet ./procedures_icd.parquet
ln -sf ./mimic/hosp/diagnoses_icd.parquet ./diagnoses_icd.parquet
ln -sf ./mimic/hosp/d_icd_procedures.parquet ./d_icd_procedures.parquet
ln -sf ./mimic/hosp/d_icd_diagnoses.parquet ./d_icd_diagnoses.parquet
ln -sf ./mimic/icu/chartevents.parquet ./chartevents.parquet
ln -sf ./mimic/icu/d_items.parquet ./d_items.parquet
```

#### Documentation

For organization, the parquet files were read in using Polars as LazyFrames. LazyFrames are memory efficient and  fast. The LazyFrames were added to a dictionary because a dictionary will hold the object's integrety while assigning names to the lazyframe.

In [ ]:
%%time
lazy_dict, df_dict = plot_dicts(10013310)

In [ ]:
data = [
    (dt.datetime(2018, 7, 17, 0, 15), dt.datetime(2018, 7, 17, 0, 30), "sleep"),
    (dt.datetime(2018, 7, 17, 0, 30), dt.datetime(2018, 7, 17, 0, 45), "eat"),
    (dt.datetime(2018, 7, 17, 0, 45), dt.datetime(2018, 7, 17, 1, 0), "work"),
    (dt.datetime(2018, 7, 17, 1, 0), dt.datetime(2018, 7, 17, 1, 30), "sleep"),
    (dt.datetime(2018, 7, 17, 1, 15), dt.datetime(2018, 7, 17, 1, 30), "eat"),
    (dt.datetime(2018, 7, 17, 1, 30), dt.datetime(2018, 7, 17, 1, 45), "work"),
]

cats = {"sleep": 1, "eat": 2, "work": 3}
colormapping = {"sleep": "C0", "eat": "C1", "work": "C2"}

verts = []
colors = []
for d in data:
    v = [
        (mdates.date2num(d[0]), cats[d[2]] - 0.4),
        (mdates.date2num(d[0]), cats[d[2]] + 0.4),
        (mdates.date2num(d[1]), cats[d[2]] + 0.4),
        (mdates.date2num(d[1]), cats[d[2]] - 0.4),
        (mdates.date2num(d[0]), cats[d[2]] - 0.4),
    ]
    verts.append(v)
    colors.append(colormapping[d[2]])

bars = PolyCollection(verts, facecolors=colors)

fig, ax = plt.subplots()
ax.add_collection(bars)
ax.autoscale()
loc = mdates.MinuteLocator(byminute=[0, 15, 30, 45])
ax.xaxis.set_major_locator(loc)
ax.xaxis.set_major_formatter(mdates.AutoDateFormatter(loc))

ax.set_yticks([1, 2, 3])
ax.set_yticklabels(["sleep", "eat", "work"])
plt.show()

### (B). ICU stays
ICU stays are a subset of ADT history. This figure shows the vitals of the patient `10001217` during ICU stays. The x-axis is the calendar time, and the y-axis is the value of the vital. The color of the line represents the type of vital. The facet grid shows the abbreviation of the vital and the stay ID. These vitals are: heart rate (220045), systolic non-invasive blood pressure (220179), diastolic non-invasive blood pressure (220180), body temperature in Fahrenheit (223761), and respiratory rate (220210). Try to create a figure similar to below:


<figure>
  <img src="https://raw.githubusercontent.com/ucla-biostat-203b/2024winter/main/hw/hw3/10001217_icu.png" style="width:600px">
</figure>

Repeat a similar visualization for the patient `10013310`.

## Problem 2. ICU stays

`icustays.csv.gz` (https://mimic.mit.edu/docs/iv/modules/icu/icustays/) contains data about Intensive Care Units (ICU) stays. The first 10 lines are:

In [ ]:
!zcat < ~/mimic/icu/icustays.csv.gz | head

### (A). Ingestion

Import `icustays.csv.gz` as a DataFrame `icustays_df`.

### (B). Summary and visualization

How many unique `subject_id`? Can a `subject_id` have multiple ICU stays? Summarize the number of ICU stays per `subject_id` by graphs.

## Problem 3. `admissions` data

Information of the patients admitted into hospital is available in `admissions.csv.gz`. See https://mimic.mit.edu/docs/iv/modules/hosp/admissions/ for details of each field in this file. The first 10 lines are

In [ ]:
!zcat < ~/mimic/hosp/admissions.csv.gz | head

### (A). Ingestion
Import `admissions.csv.gz` as a data frame `admissions_df`.

### (B). Summary and visualization

Summarize the following information by graphics and explain any patterns you see.

- number of admissions per patient
- admission hour of day (anything unusual?)
- admission minute (anything unusual?)
- length of hospital stay (from admission to discharge) (anything unusual?)

According to the MIMIC-IV documentation:

> All dates in the database have been shifted to protect patient confidentiality. Dates will be internally consistent for the same patient, but randomly distributed in the future. Dates of birth which occur in the present time are not true dates of birth. Furthermore, dates of birth which occur before the year 1900 occur if the patient is older than 89. In these cases, the patient’s age at their first admission has been fixed to 300.

## Problem 4. `patients` data
Patient information is available in `patients.csv.gz`. See https://mimic.mit.edu/docs/iv/modules/hosp/patients/ for details of each field in this file. The first 10 lines are:

In [ ]:
!zcat < ~/mimic/hosp/patients.csv.gz | head

### (A). Ingestion
Import `patients.csv.gz` (https://mimic.mit.edu/docs/iv/modules/hosp/patients/) as a data frame `patients_df`.

### (B). Summary and visaulization
Summarize variables `gender` and `anchor_age` by graphics, and explain any patterns you see.

## Problem 5. Lab results
labevents.csv.gz (https://mimic.mit.edu/docs/iv/modules/hosp/labevents/) contains all laboratory measurements for patients. The first 10 lines are

In [ ]:
!zcat < ~/mimic/hosp/labevents.csv.gz | head

`d_labitems.csv.gz` (https://mimic.mit.edu/docs/iv/modules/hosp/d_labitems/) is the dictionary of lab measurements.

In [ ]:
!zcat < ~/mimic/hosp/d_labitems.csv.gz | head

We are interested in the lab measurements of creatinine (50912), potassium (50971), sodium (50983), chloride (50902), bicarbonate (50882), hematocrit (51221), white blood cell count (51301), and glucose (50931). Retrieve a subset of `labevents.csv.gz` that only containing these items for the patients in `icustays_df`. Further restrict to the last available measurement (by `storetime`) before the ICU stay. The final `labevents_df` should have one row per ICU stay and columns for each lab measurement.
(ten columns with column names `subject_id`, `stay_id`, `Bicarbonate`, `Chloride`, ...)


_Hint_: Use the Parquet format you generated in Homework 3. For reproducibility, make `labevents.parquet` file available at the current working directory, for example, by a symbolic link.

## Problem 6. Vitals from charted events

`chartevents.csv.gz` (https://mimic.mit.edu/docs/iv/modules/icu/chartevents/) contains all the charted data available for a patient. During their ICU stay, the primary repository of a patient’s information is their electronic chart. The `itemid` variable indicates a single measurement type in the database. The `value` variable is the value measured for `itemid`. The first 10 lines of `chartevents.csv.gz` are

In [ ]:
!zcat < ~/mimic/icu/chartevents.csv.gz | head

`d_items.csv.gz` (https://mimic.mit.edu/docs/iv/modules/icu/d_items/) is the dictionary for the itemid in `chartevents.csv.gz`.

In [ ]:
!zcat < ~/mimic/icu/d_items.csv.gz | head

We are interested in the vitals for ICU patients: heart rate (220045), systolic non-invasive blood pressure (220179), diastolic non-invasive blood pressure (220180), body temperature in Fahrenheit (223761), and respiratory rate (220210). Retrieve a subset of `chartevents.csv.gz` only containing these items for the patients in `icustays_tble`. Further restrict to the first vital measurement within the ICU stay. The final `chartevents_tble` should have one row per ICU stay and columns for each vital measurement.

Hint: Use the Parquet format you generated in Homework 3. For reproducibility, make `chartevents.parquet` file available at the current working directory, for example, by a symbolic link.

## Problem 7. Putting things together
Let us create a data frame `mimic_icu_cohort` for all ICU stays, where rows are all ICU stays of adults (age at `intime` >= 18) and columns contain at least following variables

- all variables in `icustays_tble`
- all variables in `admissions_tble`
- all variables in `patients_tble`
- the last lab measurements before the ICU stay in `labevents_tble`
- the first vital measurements during the ICU stay in `chartevents_tble`
- The final `mimic_icu_cohort` should have one row per ICU stay and columns for each variable.

## Problem 8. Exploratory data analysis (EDA)
Summarize the following information about the ICU stay cohort `mimic_icu_cohort` using appropriate method:

- Length of ICU stay `los` vs demographic variables (race, insurance, marital_status, gender, age at intime)

- Length of ICU stay `los` vs the last available lab measurements before ICU stay

- Length of ICU stay `los` vs the first vital measurements within the ICU stay

- Length of ICU stay `los` vs first ICU unit

At least two plots should be created, with at least one them including multiple facets using an appropriate keyword argument. 